In [42]:
import numpy as np

def safe(array, lb=1, ub=3, tolerance=0):
    diff = np.diff(array)
    #print(f"D->{array}==>{diff}")
    if all(diff < 0):
        return diff[(diff <= -lb) & (diff >= -ub)].size >= (diff.size - tolerance)
    elif all(diff > 0):
        return diff[(diff >= lb) & (diff <= ub)].size >= (diff.size - tolerance)
    return False

def safe_brute(array, lb=1, ub=3):
    if not safe(array, lb, ub):
        for ix in range(len(array)):
            mask = np.ones(array.size, dtype=bool)
            mask[ix] = False
            aslice = array[mask]
            #print(f"Trying {aslice}")
            if safe(aslice):
                return True
    else:
        return True
    return False
    

def part1_clean(filename):
    data = np.loadtxt(filename)
    return np.sum(np.apply_along_axis(safe, 1, data))

def part1(filename):
    safeties = []
    with open(filename) as fp:
        for line in fp:
            array = np.fromstring(line.strip(), dtype=int, sep=" ")
            safeties.append(safe(array))
    return np.sum(safeties)

def part2(filename):
    safeties = []
    with open(filename) as fp:
        for line in fp:
            array = np.fromstring(line.strip(), dtype=int, sep=" ")
            #print(line.strip())
            safeties.append(safe_brute(array))
    return np.sum(safeties)
    

In [44]:
part1("sample.txt"), part1("full.txt")

(np.int64(2), np.int64(606))

In [43]:
part2("sample.txt"), part2("full.txt")

(np.int64(4), np.int64(644))

In [ ]:
def safe_part_2a(array, lb=1, ub=3):
    diff = np.diff(array)
    print("X", array, diff)
    if np.sum(diff <= 0) > np.sum(diff >= 0):
        print("NEG")
        if diff[(diff <= -lb) & (diff >= -ub)].size >= diff.size:
            print("NEG OK")
            return True
        else:
            to_rm = np.argmin((diff <= -lb) & (diff >= -ub))
            try1 = safe(np.delete(array, to_rm)) 
            try2 = True
            if to_rm + 1 < len(array):
                try2 = safe(np.delete(array, to_rm+1))
            return try1 or try2
    else:
        print("POS")
        if diff[(diff >= lb) & (diff <= ub)].size >= diff.size:
            print("POS OK")
            return True
        else:
            to_rm = np.argmax((diff >= lb) & (diff <= ub))
            try1 = safe(np.delete(array, to_rm))
            try2 = True 
            if to_rm + 1 < len(array):
                try2 = safe(np.delete(array, to_rm+1))
            return try1 or try2
    return False



def safe_part_2(array, lb=1, ub=3):
    print("==============")
    print(f"{array}")
    increasing = (array[1] - array[0]) >= 0
    skipped = False

    ix = 0

    while ix < len(array)-1:
        diff = array[ix+1] - array[ix]
        print(f"{array[ix+1]} - {array[ix]} = {diff}")
        if (not (lb <= diff <= ub) and increasing) or (not (-lb >= diff >= -ub) and not increasing):
            if not skipped and ix < len(array)-2:
                # Try skipping one
                print(f"Skipping {array[ix+1]}")
                diff2 = array[ix+2] - array[ix]
                if (not (lb <= diff2 <= ub) and increasing) or (not (-lb >= diff2 >= -ub) and not increasing):
                    print(f"{array} -> BAD")
                    return False
                else:
                    ix += 2
                    continue

            else:
                print(f"{array} -> BAD")
                return False
        ix += 1

    print(f"{array} -> OK")
    return True